# 1. Energy data

## 1.0 Startup

In [1]:
disp ('executing gpml startup script...')
mydir = fileparts (mfilename ('fullpath'));                 % where am I located
addpath (mydir)
dirs = {'cov','doc','inf','lik','mean','prior','util'};           % core folders
for d = dirs, addpath (fullfile (mydir, d{1})), end
dirs = {{'util','minfunc'},{'util','minfunc','compiled'}};     % minfunc folders
for d = dirs, addpath (fullfile (mydir, d{1}{:})), end
% addpath([mydir,'/util/sparseinv'])
pkg load statistics

executing gpml startup script...


In [2]:
dataset = 'protein'
load ('./data/protein.mat');
[ndata, ~] = size(M);
X = M(:,2:9);
Y = M(:,1);
R = randperm(ndata);        
xt = X(R(1:5730),:);
yt = Y(R(1:5730));
R(1:5730) = [];
x = X(R,:);          
y = Y(R);
n = 40000; nt = 5730;   %Ms = 50

dataset = protein


In [3]:
sf2 = 1 ; ell = 1 ; sn2 = 0.1 ; 
d = size(x,2);
hyp.cov = log([ones(d,1)*ell;sf2]); hyp.lik = log(sn2); hyp.mean = [];
opts.Xnorm = 'Y' ; opts.Ynorm = 'Y' ;
opts.Ms = 50 ;
opts.ell = ell ; opts.sf2 = sf2 ; opts.sn2 = sn2 ;
opts.meanfunc = []; opts.covfunc = @covSEard; opts.likfunc = @likGauss; opts.inffunc = @infGaussLik ;

partitionCriterion = 'kmeans'; %, 'kmeans', 'knkmeans'
opts.numOptFC = 25 ;
opts.partitionCriterion = partitionCriterion ;
[models,t_dGP_train] = aggregation_train(x,y,opts) ;
save('./res/energymodel', 'models','t_dGP_train')

In [4]:
partitionCriterion = 'kmeans'; %, 'kmeans', 'knkmeans'
opts.numOptFC = 25 ;
opts.partitionCriterion = partitionCriterion ;
s = load('./res/proteinmodel');
models = s.models;


In [5]:
criterions = {'BCM','PoE'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing protein kmeans BCM
BCM: SMSE 6.34764226, MSLL -0.7727, NLPD 2.4617
Processing protein kmeans PoE
PoE: SMSE 29.58133364, MSLL 18.0629, NLPD 21.2974


## 1.1 RBCM vs TERBCM

### 1.1.1 RBCM baseline

In [6]:
criterions = {'RBCM'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing protein kmeans RBCM
RBCM: SMSE 5.50622396, MSLL -0.9364, NLPD 2.2980


### 1.1.2 Grid q

In [7]:
partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'TERBCM'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:2.0;

for i = 1:length(partitionCriterions)
    partitionCriterion = partitionCriterions{i};
    for j = 1:length(criterions)     
        criterion = criterions{j};
        printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
        opts.partitionCriterion = partitionCriterion ;
        %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
        for q = qs
            [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
            [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
            printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
            filename = sprintf('./res/terbcm%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
            save(filename, 'SMSE','MSLL','NLPD')
        endfor
    endfor
endfor


Processing protein kmeans TERBCM
TERBCM: SMSE 5.41232640, MSLL -0.5131, NLPD 2.7213 - q   0.20
TERBCM: SMSE 5.25222299, MSLL -0.8293, NLPD 2.4051 - q   0.40
TERBCM: SMSE 5.23062591, MSLL -0.9218, NLPD 2.3126 - q   0.60
TERBCM: SMSE 5.31669529, MSLL -0.9447, NLPD 2.2897 - q   0.80
TERBCM: SMSE 5.50622396, MSLL -0.9364, NLPD 2.2980 - q   1.00
TERBCM: SMSE 5.79988650, MSLL -0.9117, NLPD 2.3227 - q   1.20
TERBCM: SMSE 6.19807833, MSLL -0.8779, NLPD 2.3566 - q   1.40
TERBCM: SMSE 6.69907763, MSLL -0.8390, NLPD 2.3955 - q   1.60
TERBCM: SMSE 7.29839621, MSLL -0.7974, NLPD 2.4370 - q   1.80
TERBCM: SMSE 7.98874818, MSLL -0.7549, NLPD 2.4795 - q   2.00


### 1.1.3 Find single q

In [8]:
criterion = 'TERBCM';

In [9]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=1000, lr=1e-9, lambda=1e6, init_q=0.8);

Optimizing q
Msize =

 Columns 1 through 11:

    800    392   1617   1764   2032   1041    153    100   1366    466    887

 Columns 12 through 22:

    367      6    570    646   1259   1717    744    355   1659    217   1052

 Columns 23 through 33:

    243    635    925    415    553    937   1447   1302    742    381    608

 Columns 34 through 44:

   1623   1316    272    670     64   1921   1681    299     15    635      8

 Columns 45 through 50:

   1263    339    419    671   1302    104

Start preprocessing...
Finish preprocessing...
Optimizing q: remain iter - 1000, q - 0.8000
grad_q_norm =  0.27939
grad_q_reg = -0.092288
Optimizing q: remain iter - 999, q - 0.8000
grad_q_norm =  1.0029
grad_q_reg = -0.27941
Optimizing q: remain iter - 998, q - 0.8000
grad_q_norm =  1.6772
grad_q_reg = -1.2599
Optimizing q: remain iter - 997, q - 0.8000
grad_q_norm = -0.65231
grad_q_reg = -0.80781
Optimizing q: remain iter - 996, q - 0.8000
grad_q_norm =  0.40781
grad_q_reg = -0.22952
Opt

grad_q_norm =  0.75833
grad_q_reg = -1.1531
Optimizing q: remain iter - 822, q - 0.8034
grad_q_norm =  0.99809
grad_q_reg = -1.2249
Optimizing q: remain iter - 821, q - 0.8034
grad_q_norm = -0.15973
grad_q_reg = -0.046836
Optimizing q: remain iter - 820, q - 0.8034
grad_q_norm = -2.0392
grad_q_reg = -1.5896
Optimizing q: remain iter - 819, q - 0.8035
grad_q_norm =  0.0016623
grad_q_reg =   -2.4271e-04
Optimizing q: remain iter - 818, q - 0.8035
grad_q_norm =  1.0861
grad_q_reg = -1.0645
Optimizing q: remain iter - 817, q - 0.8035
grad_q_norm =  0.44940
grad_q_reg = -1.2397
Optimizing q: remain iter - 816, q - 0.8035
grad_q_norm =  0.41332
grad_q_reg = -0.41188
Optimizing q: remain iter - 815, q - 0.8035
grad_q_norm =  0.0017036
grad_q_reg =   -9.2000e-04
Optimizing q: remain iter - 814, q - 0.8035
grad_q_norm =  2.4067
grad_q_reg = -1.9318
Optimizing q: remain iter - 813, q - 0.8036
grad_q_norm =  0.29965
grad_q_reg = -0.76231
Optimizing q: remain iter - 812, q - 0.8036
grad_q_norm =  

grad_q_norm =  0.83360
grad_q_reg = -2.6364
Optimizing q: remain iter - 639, q - 0.8068
grad_q_norm =  2.0259
grad_q_reg = -2.2924
Optimizing q: remain iter - 638, q - 0.8068
grad_q_norm =  1.1887
grad_q_reg = -1.8436
Optimizing q: remain iter - 637, q - 0.8068
grad_q_norm = -0.54023
grad_q_reg = -0.89965
Optimizing q: remain iter - 636, q - 0.8069
grad_q_norm =  0.23696
grad_q_reg = -0.24079
Optimizing q: remain iter - 635, q - 0.8069
grad_q_norm =  0.13485
grad_q_reg = -0.22272
Optimizing q: remain iter - 634, q - 0.8069
grad_q_norm =  0.39474
grad_q_reg = -0.71845
Optimizing q: remain iter - 633, q - 0.8069
grad_q_norm = -0.24773
grad_q_reg = -0.052490
Optimizing q: remain iter - 632, q - 0.8069
grad_q_norm = -0.30143
grad_q_reg = -0.73797
Optimizing q: remain iter - 631, q - 0.8069
grad_q_norm =    8.7077e-05
grad_q_reg =   -5.7376e-07
Optimizing q: remain iter - 630, q - 0.8069
grad_q_norm =  1.2884
grad_q_reg = -0.99756
Optimizing q: remain iter - 629, q - 0.8069
grad_q_norm =  1

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




Optimizing q: remain iter - 511, q - 0.8091
grad_q_norm =  0.76823
grad_q_reg = -1.1834
Optimizing q: remain iter - 510, q - 0.8091
grad_q_norm =  1.6584
grad_q_reg = -1.4463
Optimizing q: remain iter - 509, q - 0.8091
grad_q_norm = -0.11471
grad_q_reg = -0.024433
Optimizing q: remain iter - 508, q - 0.8091
grad_q_norm =  1.1678
grad_q_reg = -1.0384
Optimizing q: remain iter - 507, q - 0.8091
grad_q_norm =  0.10717
grad_q_reg = -0.12776
Optimizing q: remain iter - 506, q - 0.8091
grad_q_norm =  0.076875
grad_q_reg = -0.019531
Optimizing q: remain iter - 505, q - 0.8091
grad_q_norm = -0.011494
grad_q_reg = -0.0010944
Optimizing q: remain iter - 504, q - 0.8091
grad_q_norm =  3.4346
grad_q_reg = -2.7510
Optimizing q: remain iter - 503, q - 0.8092
grad_q_norm =  0.061853
grad_q_reg = -0.17865
Optimizing q: remain iter - 502, q - 0.8092
grad_q_norm =  0.73366
grad_q_reg = -0.47814
Optimizing q: remain iter - 501, q - 0.8092
grad_q_norm = -1.4208
grad_q_reg = -2.7338
Optimizing q: remain i

Optimizing q: remain iter - 328, q - 0.8123
grad_q_norm =  1.6962
grad_q_reg = -1.2040
Optimizing q: remain iter - 327, q - 0.8124
grad_q_norm =  0.0015597
grad_q_reg = -0.082369
Optimizing q: remain iter - 326, q - 0.8124
grad_q_norm =  0.51428
grad_q_reg = -1.1882
Optimizing q: remain iter - 325, q - 0.8124
grad_q_norm =  0.0083343
grad_q_reg = -0.016504
Optimizing q: remain iter - 324, q - 0.8124
grad_q_norm =  2.4597
grad_q_reg = -1.8807
Optimizing q: remain iter - 323, q - 0.8124
grad_q_norm =  0.97900
grad_q_reg = -3.1266
Optimizing q: remain iter - 322, q - 0.8125
grad_q_norm = -0.083823
grad_q_reg = -0.25017
Optimizing q: remain iter - 321, q - 0.8125
grad_q_norm =  0.83912
grad_q_reg = -0.65765
Optimizing q: remain iter - 320, q - 0.8125
grad_q_norm =  0.28357
grad_q_reg = -0.12633
Optimizing q: remain iter - 319, q - 0.8125
grad_q_norm =  1.1469
grad_q_reg = -0.95531
Optimizing q: remain iter - 318, q - 0.8125
grad_q_norm =  0.92747
grad_q_reg = -1.2219
Optimizing q: remain i

grad_q_reg = -0.73087
Optimizing q: remain iter - 144, q - 0.8155
grad_q_norm =  1.0538
grad_q_reg = -1.3466
Optimizing q: remain iter - 143, q - 0.8155
grad_q_norm =  1.3904
grad_q_reg = -0.56746
Optimizing q: remain iter - 142, q - 0.8156
grad_q_norm =  0.0088357
grad_q_reg = -0.0016319
Optimizing q: remain iter - 141, q - 0.8156
grad_q_norm =  2.5734
grad_q_reg = -1.8252
Optimizing q: remain iter - 140, q - 0.8156
grad_q_norm =  0.062230
grad_q_reg = -0.0041029
Optimizing q: remain iter - 139, q - 0.8156
grad_q_norm =  0.43147
grad_q_reg = -0.086540
Optimizing q: remain iter - 138, q - 0.8156
grad_q_norm = -2.6901
grad_q_reg = -1.3724
Optimizing q: remain iter - 137, q - 0.8156
grad_q_norm =  0.42778


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [10]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.1.4 Find multi q

In [11]:
criterion = 'TERBCM';

In [14]:
printf('Optimizing q\r\n');
iqs = q + 0.01 * rand(opts.Ms,1)
qs = mul_grad_q(models, criterion, opts, iter=1000, lr=1e-7, lambda=1e5, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

Optimizing q
iqs =

   0.82502
   0.82480
   0.82623
   0.82473
   0.82638
   0.81912
   0.82432
   0.82370
   0.82669
   0.82096
   0.82055
   0.82600
   0.82690
   0.82574
   0.82011
   0.82219
   0.82546
   0.82790
   0.82411
   0.82113
   0.82099
   0.82647
   0.81849
   0.82449
   0.82180
   0.82717
   0.82641
   0.81870
   0.82082
   0.82193
   0.82328
   0.82443
   0.82205
   0.82501
   0.82509
   0.82169
   0.82429
   0.82560
   0.82746
   0.82051
   0.82645
   0.82254
   0.82626
   0.82578
   0.81898
   0.82558
   0.82704
   0.81859
   0.82498
   0.82479

Start preprocessing...
Finish preprocessing...
Optimizing qs: remain iter - 1000
qs =

   0.82502
   0.82480
   0.82623
   0.82473
   0.82638
   0.81912
   0.82432
   0.82370
   0.82669
   0.82096
   0.82055
   0.82600
   0.82690
   0.82574
   0.82011
   0.82219
   0.82546
   0.82790
   0.82411
   0.82113
   0.82099
   0.82647
   0.81849
   0.82449
   0.82180
   0.82717
   0.82641
   0.81870
   0.82082
   0.82193
   0.82328
 

   0.82520
   0.82169
   0.82429
   0.82560
   0.82774
   0.82068
   0.82645
   0.82254
   0.82633
   0.82578
   0.81905
   0.82558
   0.82704
   0.81859
   0.82499
   0.82479

qs_dist =    1.0011e-05
Optimizing qs: remain iter - 974
qs =

   0.82547
   0.82481
   0.82623
   0.82492
   0.82656
   0.81913
   0.82432
   0.82370
   0.82669
   0.82096
   0.82063
   0.82600
   0.82690
   0.82576
   0.82011
   0.82220
   0.82557
   0.82792
   0.82411
   0.82118
   0.82099
   0.82650
   0.81849
   0.82449
   0.82182
   0.82719
   0.82641
   0.81871
   0.82089
   0.82193
   0.82334
   0.82443
   0.82205
   0.82506
   0.82520
   0.82169
   0.82429
   0.82560

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)





   0.82562
   0.82481
   0.82623
   0.82505
   0.82665
   0.81913
   0.82432
   0.82370
   0.82669
   0.82096
   0.82064
   0.82600
   0.82690
   0.82576
   0.82011
   0.82223
   0.82564
   0.82792
   0.82411
   0.82125
   0.82099
   0.82650
   0.81849
   0.82449
   0.82184
   0.82719
   0.82641
   0.81871
   0.82089
   0.82204
   0.82334
   0.82443
   0.82205
   0.82506
   0.82521
   0.82169
   0.82429
   0.82560
   0.82783
   0.82082
   0.82645
   0.82254
   0.82633
   0.82578
   0.81907
   0.82558
   0.82704
   0.81866
   0.82504
   0.82479

qs_dist =    8.4591e-05
Optimizing qs: remain iter - 958
qs =

   0.82562
   0.82481
   0.82623
   0.82505
   0.82665
   0.81913
   0.82432
   0.82370
   0.82669
   0.82096
   0.82064
   0.82600
   0.82690
   0.82576
   0.82011
   0.82225
   0.82564
   0.82792
   0.82411
   0.82125
   0.82099
   0.82650
   0.81849
   0.82449
   0.82184
   0.82719
   0.82641
   0.81871
   0.82097
   0.82204
   0.82334
   0.82443
   0.82205
   0.82506
   0.82521

   0.82100
   0.82645
   0.82254
   0.82633

...

   0.82432
   0.82370
   0.82772
   0.82104
   0.82156
   0.82600
   0.82690
   0.82606
   0.82041
   0.82336
   0.82744
   0.82810
   0.82411
   0.82231
   0.82099
   0.82678
   0.81849
   0.82452
   0.82231
   0.82732
   0.82653
   0.81929
   0.82240
   0.82235
   0.82351
   0.82444
   0.82213
   0.82634
   0.82704
   0.81890
   0.82562
   0.82479

qs_dist =    5.5544e-07
Optimizing qs: remain iter - 534
qs =

   0.83143
   0.82489
   0.82696
   0.82779
   0.82895
   0.82017
   0.82432
   0.82370
   0.82773
   0.82104
   0.82164
   0.82600
   0.82690
   0.82606
   0.82041
   0.82346
   0.82748
   0.82810
   0.82411
   0.82232
   0.82099
   0.82678
   0.81849
   0.82454
   0.82231
   0.82732
   0.82653
   0.81929
   0.82253
   0.82235
   0.82351
   0.82444
   0.82213
   0.82634
   0.82641
   0.82169
   0.82446
   0.82560
   0.83068
   0.82279
   0.82645
   0.82254
   0.82644
   0.82578
   0.81986
   0.82568
   0.82704
   0.81890
   0.82562
   0.82479

qs_dist =    4.7033e-08
Optimizin

   0.82489
   0.82702
   0.82790
   0.82903
   0.82018
   0.82432
   0.82370
   0.82791
   0.82104
   0.82174
   0.82601
   0.82690
   0.82607
   0.82041
   0.82359
   0.82753
   0.82810
   0.82411
   0.82249
   0.82099
   0.82678
   0.81849
   0.82454
   0.82235
   0.82732
   0.82653
   0.81929
   0.82271
   0.82235
   0.82351
   0.82444
   0.82213
   0.82634
   0.82666
   0.82169
   0.82446
   0.82560
   0.83072
   0.82280
   0.82645
   0.82254
   0.82644
   0.82578
   0.81987
   0.82568
   0.82704
   0.81897
   0.82565
   0.82479

qs_dist =    6.6180e-05
Optimizing qs: remain iter - 507
qs =

   0.83199
   0.82489
   0.82702
   0.82790
   0.82903
   0.82018
   0.82432
   0.82370
   0.82791
   0.82104
   0.82174
   0.82601
   0.82690
   0.82607
   0.82041
   0.82359
   0.82753
   0.82810
   0.82411
   0.82249
   0.82099
   0.82678
   0.81849
   0.82454
   0.82235
   0.82733
   0.82653
   0.81929
   0.82271
   0.82235
   0.82351
   0.82444
   0.82213
   0.82634
   0.82666
   0.82169
 

Optimizing qs: remain iter - 314
qs =

   0.83438
   0.82490
   0.82734
   0.82904
   0.82989
   0.82063
   0.82432
   0.82370
   0.82825
   0.82113
   0.82204
   0.82601
   0.82690
   0.82612
   0.82048
   0.82415
   0.82840
   0.82813
   0.82411
   0.82330
   0.82099
   0.82685
   0.81849
   0.82458
   0.82246
   0.82741
   0.82655
   0.81940
   0.82326
   0.82261
   0.82355
   0.82446
   0.82214
   0.82685
   0.82764
   0.82169
   0.82446
   0.82560
   0.83183
   0.82361
   0.82645
   0.82254
   0.82644
   0.82578
   0.82023
   0.82572
   0.82707
   0.81923
   0.82584
   0.82479

qs_dist =    1.9488e-05
Optimizing qs: remain iter - 313
qs =

   0.83438
   0.82490
   0.82734
   0.82904
   0.82990
   0.82063
   0.82432
   0.82370
   0.82825
   0.82113
   0.82204
   0.82601
   0.82690
   0.82612
   0.82048
   0.82415
   0.82840
   0.82813
   0.82411
   0.82332
   0.82099
   0.82685
   0.81849
   0.82458
   0.82246
   0.82741
   0.82655
   0.81940
   0.82326
   0.82261
   0.82355
   0.8

   0.82169
   0.82446
   0.82560
   0.83197
   0.82372
   0.82645
   0.82254
   0.82649
   0.82578
   0.82028
   0.82573
   0.82707
   0.81924
   0.82584
   0.82479

qs_dist =    3.2327e-14
Optimizing qs: remain iter - 287
qs =

   0.83461
   0.82490
   0.82748
   0.82904
   0.83002
   0.82064
   0.82432
   0.82370
   0.82829
   0.82117
   0.82204
   0.82601
   0.82690
   0.82613
   0.82048
   0.82416
   0.82865
   0.82813
   0.82411
   0.82337
   0.82099
   0.82691
   0.81849
   0.82459
   0.82248
   0.82741
   0.82655
   0.81942
   0.82330
   0.82262
   0.82355
   0.82446
   0.82214
   0.82704
   0.82764
   0.82169
   0.82446
   0.82560
   0.83197
   0.82372
   0.82645
   0.82254
   0.82649
   0.82578
   0.82028
   0.82573
   0.82707
   0.81924
   0.82584
   0.82479

qs_dist = 0
Optimizing qs: remain iter - 286
qs =

   0.83461
   0.82490
   0.82748
   0.82904
   0.83002
   0.82064
   0.82432
   0.82370
   0.82829
   0.82117
   0.82204
   0.82601
   0.82690
   0.82613
   0.82048
   0

Optimizing qs: remain iter - 87
qs =

   0.83752
   0.82490
   0.82802
   0.83018
   0.83121
   0.82091
   0.82432
   0.82370
   0.82868
   0.82118
   0.82232
   0.82601
   0.82690
   0.82618
   0.82054
   0.82497
   0.82944
   0.82814
   0.82412
   0.82400
   0.82099
   0.82693
   0.81850
   0.82460
   0.82264
   0.82744
   0.82656
   0.81954
   0.82421
   0.82292
   0.82369
   0.82449
   0.82215
   0.82781
   0.82837
   0.82169
   0.82453
   0.82560
   0.83290
   0.82464
   0.82646
   0.82254
   0.82660
   0.82578
   0.82055
   0.82575
   0.82707
   0.81925
   0.82575
   0.82707
   0.81925
   0.82601
   0.82479

qs_dist =    5.9596e-05
Optimizing qs: remain iter - 70
qs =

   0.83766
   0.82490
   0.82802
   0.83028
   0.83126
   0.82100
   0.82432
   0.82370
   0.82868
   0.82118
   0.82232
   0.82601
   0.82690
   0.82618
   0.82054
   0.82510
   0.82957
   0.82814
   0.82412
   0.82407
   0.82099
   0.82693
   0.81850
   0.82460
   0.82264
   0.82744
   0.82657
   0.81955
   0.824

In [15]:
qs
save('./res/terbcmenergydataqs', 'qs');
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

   0.83854
   0.82490
   0.82807
   0.83063
   0.83140
   0.82103
   0.82432
   0.82370
   0.82871
   0.82118
   0.82250
   0.82601
   0.82690
   0.82618
   0.82056
   0.82517
   0.82988
   0.82814
   0.82412
   0.82426
   0.82099
   0.82695
   0.81850
   0.82460
   0.82278
   0.82745
   0.82658
   0.81970
   0.82461
   0.82296
   0.82372
   0.82449
   0.82215
   0.82823
   0.82864
   0.82169
   0.82453
   0.82560
   0.83334
   0.82519
   0.82646
   0.82254
   0.82665
   0.82578
   0.82069
   0.82575
   0.82707
   0.81932
   0.82611
   0.82479

TERBCM: SMSE 5.33305484, MSLL -0.9460, NLPD 2.2885


## 1.2 GRBCM vs TEGRBCM

### 1.2.1 GRBCM baseline

In [16]:
criterions = {'GRBCM'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing protein kmeans GRBCM
GRBCM: SMSE 5.54068719, MSLL -0.9403, NLPD 2.2941


### 1.2.2 Grid q

In [17]:
criterions = {'TEGRBCM'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:2.0;

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    for q = qs
        [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
        [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
        printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
        filename = sprintf('./res/tegrbcm%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
        save(filename, 'SMSE','MSLL','NLPD')
    endfor
endfor

Processing protein kmeans TEGRBCM
TEGRBCM: SMSE 4.93896413, MSLL -0.9958, NLPD 2.2386 - q   0.20
TEGRBCM: SMSE 4.90663498, MSLL -1.0374, NLPD 2.1971 - q   0.40
TEGRBCM: SMSE 5.02037158, MSLL -1.0165, NLPD 2.2179 - q   0.60
TEGRBCM: SMSE 5.23558961, MSLL -0.9807, NLPD 2.2537 - q   0.80
TEGRBCM: SMSE 5.54068719, MSLL -0.9403, NLPD 2.2941 - q   1.00
TEGRBCM: SMSE 5.93021099, MSLL -0.8987, NLPD 2.3358 - q   1.20
TEGRBCM: SMSE 6.39916329, MSLL -0.8575, NLPD 2.3770 - q   1.40
TEGRBCM: SMSE 6.94128854, MSLL -0.8174, NLPD 2.4170 - q   1.60
TEGRBCM: SMSE 7.54864576, MSLL -0.7791, NLPD 2.4554 - q   1.80
TEGRBCM: SMSE 8.21175012, MSLL -0.7425, NLPD 2.4919 - q   2.00


### 1.2.3 Find single q

In [18]:
criterion = 'TEGRBCM';

In [27]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=1000, lr=1e-9, lambda=1e6, init_q=0.4);

Optimizing q
Start preprocessing...
Finish preprocessing...
Optimizing q: remain iter - 1000, q - 0.4000


In [28]:
q

q =  0.42232


In [29]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.2.4 Find multi q

In [21]:
criterion = 'TEGRBCM';

In [22]:
printf('Optimizing q\r\n');
iqs = q + 0.01 * rand(opts.Ms,1);
qs = mul_grad_q(models, criterion, opts, iter=5000, lr=1e-8, lambda=1e6, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

Optimizing q
Start preprocessing...
Finish preprocessing...


In [23]:
qs
save('./res/tegrbcmenergydataqs', 'qs');
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

   0.27591
   0.27314
   0.30870
   0.34625
   0.34630
   0.29724
   0.27286
   0.27477
   0.29677
   0.28244
   0.30324
   0.27366
   0.27748
   0.28474
   0.28502
   0.31239
   0.34564
   0.28257
   0.27810
   0.34530
   0.27502
   0.28497
   0.27690
   0.28210
   0.30730
   0.28324
   0.28417
   0.30051
   0.32699
   0.30890
   0.29728
   0.27497
   0.27975
   0.31987
   0.32158
   0.27245
   0.28613
   0.27495
   0.34639
   0.32625
   0.27670
   0.27859
   0.28930
   0.28007
   0.31190
   0.27954
   0.27649
   0.29461
   0.30976
   0.27314

TEGRBCM: SMSE 4.89088150, MSLL -1.0459, NLPD 2.1885


## About lambda

In [24]:
criterion = 'TEGRBCM';

In [25]:
printf('Optimizing q\r\n');
for ldk=-5:10
    ld = 10^ldk
    iqs = q + 0.01 * rand(opts.Ms,1);
    qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-3, lambda=ld, init_q=iqs);
    [mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
endfor

Optimizing q
ld =    1.0000e-05
Start preprocessing...
Finish preprocessing...
TEGRBCM: SMSE 4.90422317, MSLL -1.0264, NLPD 2.2080
ld =    1.0000e-04
Start preprocessing...
Finish preprocessing...
TEGRBCM: SMSE 4.90251730, MSLL -1.0269, NLPD 2.2075
ld =  0.0010000
Start preprocessing...
Finish preprocessing...
TEGRBCM: SMSE 4.90352214, MSLL -1.0270, NLPD 2.2074
ld =  0.010000
Start preprocessing...
Finish preprocessing...
TEGRBCM: SMSE 4.90156589, MSLL -1.0277, NLPD 2.2067
ld =  0.10000
Start preprocessing...
Finish preprocessing...
TEGRBCM: SMSE 4.90303616, MSLL -1.0274, NLPD 2.2070
ld =  1
Start preprocessing...
Finish preprocessing...
TEGRBCM: SMSE 4.89914193, MSLL -1.0323, NLPD 2.2021
ld =  10
Start preprocessing...
Finish preprocessing...
TEGRBCM: SMSE 4.88831954, MSLL -1.0543, NLPD 2.1801
ld =  100
Start preprocessing...
Finish preprocessing...
TEGRBCM: SMSE 4.96561614, MSLL -1.0563, NLPD 2.1781
ld =  1000
Start preprocessing...
Finish preprocessing...
TEGRBCM: SMSE 5.54163496, M

## 1.3 GPoE vs TEGPoE

### 1.3.1 GPoE baseline

In [5]:
criterions = {'GPoE'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing protein kmeans GPoE
GPoE: SMSE 5.32762544, MSLL -0.8720, NLPD 2.3560


### 1.3.2 Grid q

In [6]:
partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'TEGPoE'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:2.0;

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    for q = qs
        [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
        [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
        printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
        filename = sprintf('./res/tegpoe%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
        save(filename, 'SMSE','MSLL','NLPD')
    endfor
endfor

Processing protein kmeans TEGPoE
TEGPoE: SMSE 5.56106786, MSLL -0.0649, NLPD 3.1632 - q   0.20
TEGPoE: SMSE 5.50251859, MSLL -0.5848, NLPD 2.6432 - q   0.40
TEGPoE: SMSE 5.44398407, MSLL -0.7698, NLPD 2.4583 - q   0.60
TEGPoE: SMSE 5.38562976, MSLL -0.8464, NLPD 2.3817 - q   0.80
TEGPoE: SMSE 5.32762544, MSLL -0.8720, NLPD 2.3560 - q   1.00
TEGPoE: SMSE 5.27014276, MSLL -0.8693, NLPD 2.3588 - q   1.20
TEGPoE: SMSE 5.21335268, MSLL -0.8492, NLPD 2.3788 - q   1.40
TEGPoE: SMSE 5.15742281, MSLL -0.8180, NLPD 2.4100 - q   1.60
TEGPoE: SMSE 5.10251472, MSLL -0.7795, NLPD 2.4486 - q   1.80
TEGPoE: SMSE 5.04878148, MSLL -0.7360, NLPD 2.4920 - q   2.00


### 1.3.3 Find single q

In [24]:
criterion = 'TEGPoE';

In [7]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=1000, lr=1e-9, lambda=1e6, init_q=1.0001);

Optimizing q
Start preprocessing...
Finish preprocessing...
Optimizing q: remain iter - 1000, q - 1.0001


In [8]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.3.4 Find multi q

In [9]:
printf('Optimizing q\r\n');
iqs = q + 0.01* rand(opts.Ms,1);
qs = mul_grad_q(models, criterion, opts, iter=5000, lr=1e-8, lambda=1e6, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

Optimizing q
Start preprocessing...
Finish preprocessing...


In [10]:
qs
save('./res/tegpoeenergydataqs', 'qs');
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

   1.0636
   1.0195
   1.0229
   1.0284
   1.0275
   1.0173
   1.0114
   1.0115
   1.0165
   1.0151
   1.0182
   1.0187
   1.0105
   1.0192
   1.0135
   1.0277
   1.0281
   1.0177
   1.0179
   1.0274
   1.0138
   1.0113
   1.0108
   1.0113
   1.0137
   1.0135
   1.0133
   1.0135
   1.0331
   1.0169
   1.0192
   1.0165
   1.0142
   1.0243
   1.0253
   1.0102
   1.0161
   1.0185
   1.0311
   1.0279
   1.0190
   1.0163
   1.0146
   1.0168
   1.0177
   1.0170
   1.0145
   1.0148
   1.0177
   1.0139

TEGPoE: SMSE 5.31500455, MSLL -0.8760, NLPD 2.3521
